# Easy Agent Tutorial
This notebook file provide three examples of using LLM based agents with different tool sets.

Prerequisites:
- Python 3.10+
- Install required packages:
  ```bash
  pip install \
    requests \
    python-dotenv \
    langchain-community \
    langchain-openai \
    langchain \
    langgraph \
    smolagents[mcp, gradio] \
    faiss-cpu \
    tenacity \
    pyyaml
  ```

## Task Description

如README所述，该项目应用三种方案，从不同的角度实现了agentic RAG的功能。为了演示，这一次我们将会构建一个信安四大会的查询，来进行感兴趣论文的搜索以及基于题目选择合适的会议进行投稿。

## Data Preparation

In [ ]:
# 我们将使用dblp数据集来进行演示。首先下载信安四大会2025年的会议论文数据：
import requests
import json
import os

if not os.path.exists("dblp_sec_papers.json"):
    with requests.Session() as sess:
        url = "https://dblp.org/search/publ/api"
        params = {"q": "security", "format": "json"}
        tocs = {
            "ndss": "toc:db/conf/ndss/ndss2025.bht:",
            "sp": "toc:db/conf/sp/sp2025.bht:",
            "ccs": "toc:db/conf/ccs/ccs2025.bht:",
            "usenix": "toc:db/conf/uss/uss2025.bht:",
        }
        papers = []
        for k, v in tocs.items():
            response = sess.get(url, params={"q": v, "h": 1000, "format": "json"})
            data = response.json()
            data = data["result"]["hits"]["hit"]
            papers.extend(data)
        with open(f"dblp_sec_papers.json", "w", encoding="utf-8") as f:
            json.dump(papers, f, ensure_ascii=False, indent=4)
else:
    with open(f"dblp_sec_papers.json", "r", encoding="utf-8") as f:
        papers = json.load(f)
papers = [x["info"] for x in papers]
titles = [f"[{x['venue']} {x['year']}] {x['title']}" for x in papers]

在进行下一步之前，需要对环境进行一些配置，
创建一个`.env`文件，并添加以下内容：

-   示例 1：OpenAI 官方服务

```
OPENAI_API_KEY=your_openai_api_key
HTTPS_PROXY=your_proxy
MODEL_NAME=gpt-5.1
EMB_MODEL_NAME=text-embedding-3
```

In [ ]:
# 创建并保存向量数据库
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
if "db" not in globals():
    db = FAISS.from_texts(titles, OpenAIEmbeddings())
db.save_local("faiss_db")

## 方案0： 一切奇迹的始发点——传统工具调用

在开始之前，先看一下传统的工具调用大概长啥样，有怎样的优缺点

In [ ]:
import json
import logging
import re

import dotenv
import tenacity
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from openai.types import *
from openai.types.chat import *

load_dotenv()


def raw_toolcall():
    client = OpenAI()
    db = FAISS.load_local(
        "faiss_db",
        OpenAIEmbeddings(),
        allow_dangerous_deserialization=True,
    )
    tools_def = [
        {
            "type": "function",
            "name": "query_paperdb",
            "description": "accept keyword and return related information",
            "parameters": {
                "type": "object",
                "properties": {
                    "kw": {
                        "type": "string",
                        "description": "The keyword to query the database",
                    }
                },
                "required": ["kw"],
            },
        }
    ]

    def real_ask(question: str):
        from openai.types.responses.response_input_param import Message

        input_msgs: list[Message] = [
            {
                "type": "message",
                "role": "system",
                "content": f"You are a helpful research assistant. When given a question, you must first decide if you need to query the database to get relevant information. If so, use the tool 'query_paperdb' with appropriate keywords extracted from the question. After getting the information, provide a comprehensive answer based on both the retrieved information and your own knowledge.",
            },
            {
                "type": "message",
                "role": "user",
                "content": question,
            },
        ]
        resp = client.responses.create(
            model="gpt-5.1",
            tools=tools_def,
            input=input_msgs,
            tool_choice="required",
        )
        for toolcall in resp.output:
            if toolcall.type != "function_call":
                continue
            if toolcall.name == "query_paperdb":
                kw = json.loads(toolcall.arguments)["kw"]
                print(f"Keyword: {kw}")
                docs = []
                for skw in kw.split():
                    if not (skw := skw.strip()):
                        continue
                    print(f"Searching for keyword: {skw}")
                    docs.extend(db.similarity_search(skw, k=30))
                rag_result = "\n".join([doc.page_content for doc in docs])
                input_msgs.append(toolcall)
                input_msgs.append(
                    {
                        "type": "function_call_output",
                        "call_id": toolcall.call_id,
                        "output": str(rag_result),
                    }
                )
                print(f"{kw=} appended.")
        if input_msgs[-1]["type"] == "function_call_output":
            resp = client.responses.create(
                model="gpt-5.1",
                input=input_msgs,
                # tools=tools_def,
                stream=True,
            )
            for chunk in resp:
                if chunk.type == "response.output_text.delta":
                    print(chunk.delta, end="", flush=True)
            print()
        else:
            print(resp.output_text)

    while True:
        inp = input("=> ")
        if not inp.strip():
            break
        real_ask(inp)


raw_toolcall()

### 传统工具调用的优缺点

- 优点：直接用模型原生的 function/tool 调用协议，链路短、开销低，JSON Schema 参数校验清晰。
- 优点：可以精确控制何时调用工具、使用 `tool_choice` 等参数强制执行，消息格式透明、便于调试和流式输出。
- 优点：依赖少，不绑框架，易于插入到现有服务或与其他编排层组合。
- 缺点：需要手写对话状态管理、工具输入输出拼接，容易出错且样板代码多。
- 缺点：缺少自动规划/多步推理、重试、fallback 等封装能力，复杂流程要自行实现。
- 缺点：与特定模型/协议耦合，换提供方或多模型时需适配；安全性与数据清洗（如反序列化、去重）也要自管。

## 方案1： SmolAgent::CodeAgent

SmolAgent::CodeAgent 拥有两种提示模板，使用`use_structured_outputs_internally`参数进行控制。
- 核心循环：两条线路都遵循多步推理，但输出格式不同。
  - 标准版（`code_agent.yaml`，`use_structured_outputs_internally=False`）：每步输出 Thought→Code→Observation，代码必须包裹在自定义 `{{code_block_opening_tag}}`/`{{code_block_closing_tag}}` 中；中间结果用 `print` 进入 Observation，最终用 `final_answer` 收束。
  - 结构化版（`structured_code_agent.yaml`，`use_structured_outputs_internally=True`）：每步输出固定 JSON `{ "thought": "...", "code": "..." }`，默认不需要自定义 code_block 标签；解析稳定、便于日志/回放。
- 规划 scaffold：两者都要求先做 facts survey + high-level plan（initial_plan / update_plan），但标准版在文本里更强调「分步打印、避免链式依赖」。
- 工具调用与链式策略：
  - 标准版明确区分“有 JSON schema 的工具可以链式调用，非结构化工具应先 `print` 再下一步用”，并警示不要重做相同参数的调用、不要用工具名做变量名。
  - 结构化版主要提供最小约束（定义变量再用、参数名显式），输出以 JSON 承载 Thought 和 Code，便于上层程序直接消费。
- 执行与可读性：
  - 标准版输出包含 Thought/Code/Observation 叙事，适合人工旁观调试、流式展示和教学场景。
  - 结构化版输出紧凑、机器友好，适合日志解析、监控、对接编排器或二次路由。
- 选择建议：
  - 需要确定性结构、便于程序解析/回放/审计时，用 `use_structured_outputs_internally=True`（结构化版）。
  - 需要人类可读的逐步对话、希望看到 code_block 包裹和 Observation 明细，或想依赖模板中的链式提示时，用 `use_structured_outputs_internally=False`（标准版）。
- 常见踩坑提示：
  - 标准版务必保持自定义 code_block 标签，否则解析失败；非结构化工具调用不要在同一块紧接依赖下一工具输出。
  - 结构化版的 JSON 输出中请将可执行代码写在 `code` 字段，仍需显式 `final_answer(...)` 收口；确保不要把工具名当变量名。
- 性能与上下文：标准版提示体积更大，可能略增 token 开销；结构化版更紧凑，节省上下文。

In [ ]:
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from smolagents import CodeAgent, OpenAIServerModel, tool

load_dotenv()

db = FAISS.load_local(
    "faiss_db",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)

sys_prompt = """
You are a helpful research assistant.
When given a question, you must query the database to get relevant information.
Use the tools with appropriate arguments derived from the question.
After getting the information, provide a comprehensive answer based on both the retrieved information.
Output the final answer in markdown wrapped in final_answer().
""".strip()


@tool
def query_paperdb(kw: str) -> str:
    """
    Accept keywords and return related paper titles, multiple keywords should be separated by '|'.
    This tool should be called before output any realworld-related information.

    Args:
        kw (str): The keyword to query the database
    """
    docs = []
    for skw in kw.split("|"):
        if not (skw := skw.strip()):
            continue
        print(f"Searching for keyword: {skw}")
        docs.extend(db.similarity_search(skw, k=10))
    rag_result = "\n".join([doc.page_content for doc in docs])
    return rag_result


agent = CodeAgent(
    model=OpenAIServerModel("gpt-5.1"),
    tools=[query_paperdb],
    stream_outputs=True,
    # use_structured_outputs_internally=True, # True for structured_code_agent.yaml, False for code_agent.yaml
)

# from Gradio_UI import GradioUI

# GradioUI(agent).launch()

_ = agent.run(f"{sys_prompt}\n\n{input("=> ")}", return_full_result=True)

## 方案2： LangGraph

以下代码块用 LangGraph 重新实现「检索→工具调用→回答」的代理流程，强调显式状态机与可视化路由的优势：



- 基础资源：启动时加载 `.env`，用 `FAISS.load_local` + `OpenAIEmbeddings` 复用前面构建的向量库，确保跨会话复现检索结果。

- 系统提示：`sys_prompt` 要求回答前必须调用检索工具，并用 `final_answer(...)` 收口；保持与前两种方案一致，便于对比 token 与行为。

- 工具实现：`@tool` 装饰的 `query_paperdb` 按 `|` 分割多关键词，每个关键词检索 top-10，stdout 打印检索日志，返回拼接文本作为 RAG 证据。

- 模型与绑定：`ChatOpenAI(model="gpt-5.1", verbose=True)` 绑定工具得到 `chat_with_tools`，LangGraph 自动根据模型给出的 `tool_calls` 决定是否进入工具节点。

- 状态与路由：`AgentState` 仅维护 `messages`（用 `add_messages` 合并），`StateGraph` 定义两个节点：
  - `assistant`：调用大模型，可能产出工具调用或直接回答。
  - `tools`：`ToolNode` 执行 `query_paperdb` 并把结果写回消息。
 通过 `tools_condition` 条件边在两节点间循环，直到模型不再请求工具。

- 编译与运行：`builder.compile()` 得到 `agent`，随后用 `SystemMessage+HumanMessage` 作为初始消息调用 `agent.invoke`。执行结束后按顺序打印每段回复，便于观察模型与工具交替的对话片段。

- 适用场景：当需要显式控制多轮工具路由、插入额外节点（如过滤、重排序、裁剪上下文）或做流程可视化/监控时，LangGraph 方案更易扩展；相较裸工具调用（方案0）和 SmolAgent（方案1），它在可编排性与可观测性上更强。

In [ ]:
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools import tool

load_dotenv()

db = FAISS.load_local(
    "faiss_db",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)

sys_prompt = """
You are a helpful research assistant.
When given a question, you must query the database to get relevant information.
Use the tools with appropriate arguments derived from the question.
After getting the information, provide a comprehensive answer based on both the retrieved information.
Output the final answer in markdown wrapped in final_answer().
""".strip()


@tool
def query_paperdb(kw: str) -> str:
    """
    Accept keywords and return related paper titles, multiple keywords should be separated by '|'.
    This tool should be called before output any realworld-related information.

    Args:
        kw (str): The keyword to query the database
    """
    docs = []
    for skw in kw.split("|"):
        if not (skw := skw.strip()):
            continue
        print(f"Searching for keyword: {skw}")
        docs.extend(db.similarity_search(skw, k=10))
    rag_result = "\n".join([doc.page_content for doc in docs])
    return rag_result


from typing import Annotated, TypedDict

from langchain_core.messages import AIMessage, AnyMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

chat = ChatOpenAI(model="gpt-5.1", verbose=True)
chat_with_tools = chat.bind_tools([query_paperdb])


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }


builder = StateGraph(AgentState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode([query_paperdb]))
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

agent = builder.compile()

messages = [
    SystemMessage(content=sys_prompt),
    HumanMessage(content=input("=> ")),
]

response = agent.invoke({"messages": messages})

for idx, part in enumerate(response["messages"]):
    print(f"Part {idx}: \n{part.content}\n\n")

## 方案3： CodeAgent + MCP (Model Context Protocol)

这一节展示如何通过 MCP 将外部工具集合挂载给 SmolAgent 的 CodeAgent，并保持与前面方案一致的“先检索后回答”规范。使用前请先在本目录启动 `mcp-1.py` 以提供本地 MCP 服务器。



- 启动服务：在终端运行 `python mcp-1.py`，保持进程存活，默认监听 `http://127.0.0.1:8000/mcp`。

- 工具获取：`ToolCollection.from_mcp` 以 streamable-http 方式连接服务器，`trust_remote_code=True` 允许加载远端工具实现，`structured_output=False` 让工具返回原始文本。

- 代理配置：创建 `CodeAgent(model=OpenAIServerModel("gpt-5.1"), tools=[*tool_collection.tools], stream_outputs=True, use_structured_outputs_internally=True)`，沿用 `sys_prompt` 强制先调用工具，再用 `final_answer(...)` 收口。

- 运行流程：读取用户输入后直接 `agent.run`，中间的工具调用与结果交由 MCP 服务提供，实现与本地代码解耦、便于统一暴露多工具。

- 适用场景：需要集中托管工具、复用跨项目/多语言工具链，或将工具部署为独立服务时，MCP 能作为标准化桥梁，保持模型侧最小改动。

In [ ]:
from smolagents import CodeAgent, OpenAIServerModel, ToolCollection

from dotenv import load_dotenv

load_dotenv()

sys_prompt = """
You are a helpful research assistant.
When given a question, you must query the database to get relevant information.
Use the tools with appropriate arguments derived from the question.
After getting the information, provide a comprehensive answer based on both the retrieved information.
* Output the final answer in markdown wrapped in final_answer().
""".strip()

with ToolCollection.from_mcp(
    {"url": "http://127.0.0.1:8000/mcp", "transport": "streamable-http"},
    trust_remote_code=True,
    structured_output=False,
) as tool_collection:
    agent = CodeAgent(
        model=OpenAIServerModel("gpt-5.1"),
        tools=[*tool_collection.tools],
        stream_outputs=True,
        use_structured_outputs_internally=True,
    )

    # from Gradio_UI import GradioUI

    # GradioUI(agent).launch()

    agent.run(f"{sys_prompt}\n\n{input('=> ')}")

## Conclusion

- 数据基座：从 dblp 拉取信安四大会 2025 TOC，用 `OpenAIEmbeddings` 构建并持久化 `FAISS` 向量库，后续各方案共享。

- 方案0（原生 tool call）：直接用 OpenAI `responses.create` + JSON Schema 工具。链路最短、调试透明、可强制 `tool_choice`；但对话状态管理、重试、去重、输出都要手写。

- 方案1（SmolAgent::CodeAgent）：封装规划+多步推理，`use_structured_outputs_internally` 决定输出格式（叙事式 vs JSON）。自带 `@tool` 装饰、流式打印，适合想要自动规划、教学演示或快速接 UI 的场景。

- 方案2（LangGraph）：用显式状态机把 LLM 节点与 `ToolNode` 串起来，`tools_condition` 控制循环；便于插入过滤/重排/裁剪等治理节点，可视化和监控更友好，适合可编排性与合规要求高的流水线。

- 方案3（CodeAgent + MCP）：工具通过 MCP 服务集中托管，前端代理几乎零改动即可复用跨项目/多语言工具；适合团队内统一工具治理、远程扩展或把工具独立部署成服务。

- 选型指南：

  - 快速内嵌或最小依赖：选方案0。

  - 需要自动规划、流式展示或教学：选方案1。

  - 流程复杂、需可视化/治理节点：选方案2。

  - 工具要集中化、跨项目共享或远程调用：选方案3。